# DATA PREPARATION

In [ ]:
# TO TEACHING STAFF: 
## change the object_id field here
object_id = 1 # 1 = Lausanne

The following code will get (and save locally) all the data you will need to run the journey finder in the selected region.

Note that we have already provided you with the Lausanne data (objectid = 1).

## 1. Setup

In [ ]:
import os
import sys
import warnings
import pandas as pd
from pyhive import hive
from ipywidgets import widgets
from IPython.display import display

warnings.simplefilter(action='ignore', category=UserWarning)

default_db = 'com490'
hive_server = os.environ.get('HIVE_SERVER','iccluster080.iccluster.epfl.ch:10000')
hadoop_fs = os.environ.get('HADOOP_DEFAULT_FS','hdfs://iccluster067.iccluster.epfl.ch:8020')
username  = os.environ.get('USER', 'anonym')
(hive_host, hive_port) = hive_server.split(':')

conn = hive.connect(
    host=hive_host,
    port=hive_port,
    username=username
)

# create cursor
cur = conn.cursor()

print(f"hadoop hdfs URL is {hadoop_fs}")
print(f"your username is {username}")
print(f"you are connected to {hive_host}:{hive_port}")

In [ ]:
# Query the tables in default_db
query = f"""
    SHOW TABLES IN {default_db}
"""

cur.execute(query)
default_db_tables = cur.fetchall()
default_db_tables

## 2. Enable support for ESRI UDF

In [ ]:
# Query JAR files currently available in the session
cur.execute("LIST JARS")
cur.fetchall()

In [ ]:
# List the contents of the directory '/data/jars' in HDFS -> JAR files to be added
cur.execute("DFS -ls /data/jars")
results = cur.fetchall()

# Construct full HDFS paths for each JAR file
jar_paths = [os.path.join(hadoop_fs, x[0].split()[-1][1:]) for x in results[1:]]
jar_paths

In [ ]:
# Construct and execute query to add all the JAR files
add_jars_query = "ADD JARS\n\t" + "\n\t".join(jar_paths)
cur.execute(add_jars_query)

In [ ]:
 # Check that JAR files were added
cur.execute("LIST JARS")
cur.fetchall()

In [ ]:
cur.execute("CREATE TEMPORARY FUNCTION ST_Point AS 'com.esri.hadoop.hive.ST_Point'")
cur.execute("CREATE TEMPORARY FUNCTION ST_Distance AS 'com.esri.hadoop.hive.ST_Distance'")
cur.execute("CREATE TEMPORARY FUNCTION ST_SetSRID AS 'com.esri.hadoop.hive.ST_SetSRID'")
cur.execute("CREATE TEMPORARY FUNCTION ST_GeodesicLengthWGS84 AS 'com.esri.hadoop.hive.ST_GeodesicLengthWGS84'")
cur.execute("CREATE TEMPORARY FUNCTION ST_LineString AS 'com.esri.hadoop.hive.ST_LineString'")
cur.execute("CREATE TEMPORARY FUNCTION ST_AsBinary AS 'com.esri.hadoop.hive.ST_AsBinary'")
cur.execute("CREATE TEMPORARY FUNCTION ST_PointFromWKB AS 'com.esri.hadoop.hive.ST_PointFromWKB'")
cur.execute("CREATE TEMPORARY FUNCTION ST_GeomFromWKB AS 'com.esri.hadoop.hive.ST_GeomFromWKB'")
cur.execute("CREATE TEMPORARY FUNCTION ST_Contains AS 'com.esri.hadoop.hive.ST_Contains'")

## 3. Region Selection

The following query selects all the stops in the specified region.
We decided to save them both on a file, to use it in the ui, and also in a table, to query it later. 

In [ ]:
# NOTE: OBJECT_ID SHOULD BE MODIFIED AT TOP OF FILE!

In [ ]:
query = f"""
    CREATE TABLE IF NOT EXISTS {username}.sbb_stops_in_region(
        stop_id        string,
        stop_name      string,
        stop_lat       double,
        stop_lon       double
    )
    STORED AS ORC
"""
cur.execute(query)

In [ ]:
%%time
cur.execute(f"""
    INSERT OVERWRITE TABLE {username}.sbb_stops_in_region
    SELECT a.stop_id, a.stop_name, a.stop_lat, a.stop_lon
    FROM {default_db}.sbb_orc_stops a JOIN {default_db}.geo_shapes b
    WHERE b.objectid={object_id}
    AND ST_Contains(b.geometry, ST_Point(stop_lon,stop_lat))
""")

In [ ]:
%%time
query = f"""
    SELECT 
        a.stop_id as stop_id,
        a.stop_name as stop_name,
        a.stop_lat as stop_lat,
        a.stop_lon as stop_lon
    FROM {default_db}.sbb_orc_stops a JOIN {default_db}.geo_shapes b
    WHERE b.objectid={object_id}
    AND ST_Contains(b.geometry, ST_Point(stop_lon,stop_lat))
"""

region_stops = pd.read_sql(query, conn)
region_stops.head()

data_path = "data/"
if not os.path.exists(data_path):
    os.makedirs(data_path)
region_stops.to_csv(data_path + "stops.csv", index = False)

## 4. Compute Footpaths

The following query selects all the pairs of stops with a distance of at most 500m and then compute the walking distances between them.

In [ ]:
query = f"""
    SELECT a.stop_id as stop_id_a, b.stop_id as stop_id_b, ST_GeodesicLengthWGS84(ST_SetSRID(ST_LineString(a.stop_lon,a.stop_lat,b.stop_lon,b.stop_lat), 4326)) as distance
    FROM {username}.sbb_stops_in_region a
    JOIN {username}.sbb_stops_in_region b
    WHERE a.stop_id != b.stop_id
    AND ST_GeodesicLengthWGS84(
            ST_SetSRID(ST_LineString(a.stop_lon,a.stop_lat,b.stop_lon,b.stop_lat), 4326)
        ) < 500
"""

df_near_stops = pd.read_sql(query, conn)
df_near_stops = df_near_stops.drop_duplicates()
df_near_stops.head()

In [ ]:
# Add duration
walk_speed = 50/60 # (50m/1min) m/s 
df_near_stops['duration'] = df_near_stops['distance'] / walk_speed

df_reversed = df_near_stops.rename(columns={'stop_id_a': 'stop_id_a_noconflict', 'stop_id_b': 'stop_id_b_noconflict'})
df_reversed = df_reversed.rename(columns={'stop_id_a_noconflict': 'stop_id_b', 'stop_id_b_noconflict': 'stop_id_a'})
df_near_stops = pd.concat([df_near_stops, df_reversed], ignore_index=True)

df_near_stops.head()

In [ ]:
%%time
data_path = "data/"
if not os.path.exists(data_path):
    os.makedirs(data_path)
df_near_stops.to_csv(data_path + "footpaths.csv", index = False)

## 4. Timetable

This query retrieves all trips and their corresponding details for each stop within the specified region.

In [ ]:
query = f"""
    SELECT
        times.trip_id AS trip_id,
        times.stop_id AS stop_id,
        times.arrival_time AS arrival_time,
        times.departure_time AS departure_time,
        trips.monday as monday,
        trips.tuesday as tuesday,
        trips.wednesday as wednesday,
        trips.thursday as Thursday,
        trips.friday as friday,
        trips.saturday as saturday,
        trips.sunday as sunday,
        trips.start_date as start_date,
        trips.end_date as end_date
    FROM 
        {default_db}.sbb_orc_stop_times times
    JOIN (
        SELECT *
        FROM {default_db}.sbb_orc_trips trips
        JOIN {default_db}.sbb_orc_calendar calendar
        ON trips.service_id = calendar.service_id
    ) trips
    ON times.trip_id = trips.trip_id
    JOIN 
        {username}.sbb_stops_in_region region
    ON times.stop_id = region.stop_id
"""
# Print statements for validation
df_stops = pd.read_sql(query, conn)
print('stop_times found:', df_stops.shape[0])
print('random samples:')
df_stops.sample(5)

In [ ]:
def construct_schedule_dataframe(schedule):
    """
    Constructs a dataframe of stop-to-stop connections from a transit schedule within a 2-hour window from the start time.

    Parameters:
    - schedule: DataFrame with columns ['departure_time', 'arrival_time', 'trip_id', 'stop_id']
    - start_time: start time of the trip as a string in HH:MM:SS format

    Returns:
    - df: DataFrame with columns ['Stop_a', 'Stop_b', 'Trip_id', 'Departure_time', 'Arrival_time']
    """

    schedule['arr_time'] = pd.to_timedelta(schedule['arrival_time']).dt.total_seconds()
    schedule['dep_time'] = pd.to_timedelta(schedule['departure_time']).dt.total_seconds()
    schedule.sort_values(by=['trip_id', 'arr_time'])

    timetable_obs = []
    schedule_n = len(schedule)
    for i in range(schedule_n-1):
        if i % 50000 == 0:
            print('fraction done: ', i/schedule_n)
        if schedule.iloc[i]['trip_id'] == schedule.iloc[i+1]['trip_id']:
            timetable_obs.append([schedule.iloc[i]['stop_id'], schedule.iloc[i+1]['stop_id'], schedule.iloc[i]['dep_time'], schedule.iloc[i+1]['arr_time'], schedule.iloc[i]['trip_id'], schedule.iloc[i]['monday'], schedule.iloc[i]['tuesday'], schedule.iloc[i]['wednesday'], schedule.iloc[i]['thursday'], schedule.iloc[i]['friday'], schedule.iloc[i]['saturday'], schedule.iloc[i]['sunday']])
    timetable = pd.DataFrame(timetable_obs, columns=['dep_stop', 'arr_stop', 'dep_time', 'arr_time', 'trip_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'])
    timetable['connection_id'] = timetable.groupby('trip_id').cumcount() + 1
    timetable['connection_id'] = timetable['connection_id'].astype(str) + '_' + timetable['trip_id']
    timetable = timetable.sort_values(by='arr_time')

    return timetable

# Construct the dataframe (see function description)
timetable = construct_schedule_dataframe(df_stops)
timetable.head(5)

In [ ]:
%%time
data_path = "data/"
if not os.path.exists(data_path):
    os.makedirs(data_path)
timetable.to_csv(data_path + "timetable.csv", index = False)

## 5. Stop ID Handling

Since stops id inside isdaten table and the one inside the expected stops' tables may be different, we create a table containing the match between them using their coordinates.

In [ ]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, DoubleType
import getpass

In [ ]:
spark = SparkSession.builder.appName('final-project-{0}'.format(getpass.getuser())).getOrCreate()

sc = spark.sparkContext
conf = sc.getConf()

In [ ]:
isdaten_stops_df = spark.read.csv('/data/sbb/csv/timetables/stops', header=True)\
                .select(['stop_id','stop_lat', 'stop_lon'])\
                .dropDuplicates(['stop_id'])\
                .withColumnRenamed("stop_id", "stops_stop_id")\
                .withColumn("stop_lat", col("stop_lat").cast(DoubleType()))\
                .withColumn("stop_lon", col("stop_lon").cast(DoubleType()))
isdaten_stops_df.show(5)

In [ ]:
data_path = "data/"
isdaten_stops_df.toPandas().to_csv(data_path + "isdaten_stops_full.csv", index = False)

In [ ]:
import json
import pandas as pd
import numpy as np

a = pd.read_csv("data/stops.csv")
a = a.rename(columns={"stop_id":"stop_id_a"})
b = pd.read_csv("data/isdaten_stops_full.csv")
b = b.rename(columns={"stops_stop_id":"stop_id_b"})

# see: https://en.wikipedia.org/wiki/Haversine_formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

closest_stops = {}
for _, row_a in a.iterrows():
    distances = haversine(row_a['stop_lat'], row_a['stop_lon'], b['stop_lat'], b['stop_lon'])
    closest_stop_id = b.loc[np.argmin(distances), 'stop_id_b']
    closest_stops[row_a['stop_id_a']] = closest_stop_id
    
df = pd.DataFrame.from_dict(closest_stops, orient='index', columns=['isdaten_stop_id']).reset_index()
df.rename(columns={'index': 'journey_stop_id'}, inplace=True)
df.to_csv('data/stop_matching.csv')

In [ ]:
spark_df = spark.createDataFrame(df)
spark_df.printSchema()

In [ ]:
hdfs_path = f"/user/{username}/stop_matching.csv"
spark_df.write.csv(hdfs_path, mode="overwrite", header="True")